In [25]:
import pandas as pd
import pywikibot as pwb

In [26]:
frame_data = pd.read_table("sampling_frame.tsv")
frame_data.head()

,user_name,home_wiki,global_edits,wiki,edits
0,! Bikkit !,dewiki,1658.6,dewiki,1435.0
1,! Bikkit !,dewiki,1658.6,enwiki,190.0
2,! Bikkit !,dewiki,1658.6,commonswiki,32.0
3,! Bikkit !,dewiki,1658.6,frwiki,1.0
4,! Bikkit !,dewiki,1658.6,wikidatawiki,0.6


In [27]:
per_wiki = frame_data[["user_name", "wiki", "edits"]]
per_wiki = per_wiki.pivot("user_name", "wiki")
per_wiki.columns = per_wiki.columns.droplevel()
per_wiki.head()

wiki,abwiki,acewiki,adywiki,afwiki,afwikibooks,afwikiquote,afwiktionary,akwiki,alswiki,amwiki,...,zh_yuewiki,zhwiki,zhwikibooks,zhwikinews,zhwikiquote,zhwikisource,zhwikivoyage,zhwiktionary,zuwiki,zuwiktionary
user_name,,,,,,,,,,,,,,,,,,,,,
! Bikkit !,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!KrzysiekBu!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Manihiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Silent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!minmi73!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
all_wikis = frame_data[["user_name", "home_wiki", "global_edits"]]
grouped = all_wikis.groupby("user_name")
all_wikis = grouped.first().reset_index()
all_wikis.head()

,user_name,home_wiki,global_edits
0,! Bikkit !,dewiki,1658.6
1,!KrzysiekBu!,plwiki,2463.6
2,!Manihiki,itwikibooks,648.0
3,!Silent,ptwiki,26321.5
4,!minmi73!,jawiki,64.0


In [29]:
frame = all_wikis.join(per_wiki, on = "user_name")
frame.head()

,user_name,home_wiki,global_edits,abwiki,acewiki,adywiki,afwiki,afwikibooks,afwikiquote,afwiktionary,...,zh_yuewiki,zhwiki,zhwikibooks,zhwikinews,zhwikiquote,zhwikisource,zhwikivoyage,zhwiktionary,zuwiki,zuwiktionary
0,! Bikkit !,dewiki,1658.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!KrzysiekBu!,plwiki,2463.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,!Manihiki,itwikibooks,648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,!Silent,ptwiki,26321.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,!minmi73!,jawiki,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
strata = pd.read_table("strata_definitions.tsv")
strata.head()

,code,audience,home_wiki,also_active_enwiki,min_sample,sample,population
0,01-AEAR,AE,arwiki,0,150,225,NaN
1,02-AEZH,AE,zhwiki,0,150,225,NaN
2,03-AEFR,AE,frwiki,0,150,225,NaN
3,04-AEES,AE,eswiki,0,375,563,NaN
4,05-AERU,AE,ruwiki,0,150,225,NaN


In [31]:
# Tidy data
strata["home_wiki"] = strata["home_wiki"].apply(str.strip)
strata.loc[
    strata['home_wiki'] == "other projects",
    "home_wiki"
] = "other"

In [32]:
wps_by_group = pd.read_table("wikipedias_by_group.tsv")

#Remove when fixed in data
wps_by_group.loc[wps_by_group["site_key"] == "ruwiki", "group"] = "ruwiki"
wps_by_group.head()

,language,site_key,group
0,Afar,aawiki,ssa
1,Abkhazian,abwiki,mena
2,Acehnese,acewiki,asia
3,Adyghe,adywiki,cee
4,Afrikaans,afwiki,ssa


In [33]:
# Map each group to its component wikis
wiki_groups = {group: None for group in set(defs["home_wiki"])}

regions = ["ssa", "mena", "asia", "cee", "weur"]

for group in regions:
    wiki_groups[group] = set(wps_by_group.query("group == @group")["site_key"])

all_projects = set(frame_data["wiki"])

wiki_groups["other"] = all_projects - wiki_groups.keys() - wiki_groups["ssa"] - wiki_groups["mena"] \
    - wiki_groups["asia"] - wiki_groups["cee"] - wiki_groups["weur"]
    
for key, val in wiki_groups.items():
    if val is None:
        wiki_groups[key] = set([key])

In [41]:
queries = []

for stratum in strata.itertuples(index = False):
    if stratum.audience == "AE":
        query = "global_edits < 600"
    else:
        query = "global_edits >= 600"
    
    query += (" & home_wiki.isin({})".format(list(wiki_groups[stratum.home_wiki])))
    
    if stratum.home_wiki != "enwiki":
        if stratum.also_active_enwiki:
            query += " & enwiki >= 30"
        else:
            query += " & (enwiki < 30 | enwiki.isnull())"
    
    queries.append(query)

for query in strata["query"][:5]:
    print(query)

global_edits < 600 & home_wiki.isin(['arwiki']) & (enwiki < 30 | enwiki.isnull())
global_edits < 600 & home_wiki.isin(['zhwiki']) & (enwiki < 30 | enwiki.isnull())
global_edits < 600 & home_wiki.isin(['frwiki']) & (enwiki < 30 | enwiki.isnull())
global_edits < 600 & home_wiki.isin(['eswiki']) & (enwiki < 30 | enwiki.isnull())
global_edits < 600 & home_wiki.isin(['ruwiki']) & (enwiki < 30 | enwiki.isnull())


In [48]:
def sample_stratum(stratum_query, sample_size):
    """
    returns a tuple consisting of (1) the population count for that subgroup and
    (2) a list of the sampled users in MassMessage format 
    """ 
    stratum = frame.query(stratum_query)
    stratum_size = len(stratum)
    
    if sample_size == 0:
        sample = None
    elif sample_size > stratum_size:
        sample = stratum[["user_name", "home_wiki"]]
    else:
        sample = stratum.sample(n = sample_size)[["user_name", "home_wiki"]]
        
    if sample is None:
        sample_list = None
    else:
        sample_list = [
            i for i in sample.itertuples(index = False, name = "user")
        ]

    return (stratum_size, sample_list)

In [49]:
results = [[], []]

for row in strata.itertuples(index = False):
    size, sample = sample_stratum(row.query, row.sample)
    results[0].append(size)
    results[1].append(sample)
    
strata["population"] = results[0]
strata["sampled_users"] = results[1]

In [56]:
strata.head(n = 10)

,code,audience,home_wiki,also_active_enwiki,min_sample,sample,population,query,sampled_users
0,01-AEAR,AE,arwiki,0,150,225,1077,global_edits < 600 & home_wiki.isin(['arwiki']...,"[(Badeee3, arwiki), (Yazedmaki, arwiki), (Agar..."
1,02-AEZH,AE,zhwiki,0,150,225,2870,global_edits < 600 & home_wiki.isin(['zhwiki']...,"[(林佳郁, zhwiki), (Drice, zhwiki), (Mico121, zhw..."
2,03-AEFR,AE,frwiki,0,150,225,5539,global_edits < 600 & home_wiki.isin(['frwiki']...,"[(Nicowboy, frwiki), (Dekerdrel, frwiki), (Coy..."
3,04-AEES,AE,eswiki,0,375,563,5023,global_edits < 600 & home_wiki.isin(['eswiki']...,"[(Samoano23, eswiki), (R.RicardoRamirez.Vzla, ..."
4,05-AERU,AE,ruwiki,0,150,225,4268,global_edits < 600 & home_wiki.isin(['ruwiki']...,"[(Shivani, ruwiki), (VadimKat, ruwiki), (Monte..."
5,06-AEDE,AE,dewiki,0,150,225,6847,global_edits < 600 & home_wiki.isin(['dewiki']...,"[(FrontOffice, dewiki), (Alexander Jörk, dewik..."
6,07-AEPT,AE,ptwiki,0,375,563,1995,global_edits < 600 & home_wiki.isin(['ptwiki']...,"[(Allan CZ, ptwiki), (LucasIII3, ptwiki), (Mar..."
7,08-AENL,AE,nlwiki,0,150,225,1223,global_edits < 600 & home_wiki.isin(['nlwiki']...,"[(Appelsenperenenzo, nlwiki), (Digi-garage, nl..."
8,09-AEIT,AE,itwiki,0,150,225,3146,global_edits < 600 & home_wiki.isin(['itwiki']...,"[(Stardust 1992, itwiki), (Dnilor, itwiki), (C..."
9,10-AEJA,AE,jawiki,0,150,225,5635,global_edits < 600 & home_wiki.isin(['jawiki']...,"[(雨安居, jawiki), (飴ペロ太, jawiki), (Atlantic Aven..."


The users' projects need to be in domain form, so get a table mapping site keys to site domains:
```
select 
site_global_key as site_key,
substring(reverse(site_domain), 2) as site_domain
from enwiki.sites;
```

In [24]:
domains = pd.read_table("site_domains.tsv", index_col = 0)

def lookup_domain(site_key):
    return domains.loc[site_key, "site_domain"]

domains.head()

,site_domain
site_key,
aawiki,aa.wikipedia.org
aawiktionary,aa.wiktionary.org
aawikibooks,aa.wikibooks.org
abwiki,ab.wikipedia.org
abwiktionary,ab.wiktionary.org


In [57]:
for row in strata.itertuples(index = False):
    if row.sampled_users:
        text = ""
        for user in row.sampled_users:
            text += "* {{{{target | user = {} | site = {}}}}}\n".format(user[0], lookup_domain(user[1]))
        site = pwb.Site()
        page = pwb.Page(site, "Community Engagement Insights/MassMessages/Lists/2016/" + row.code)
        page.text = text
        page.save(u"Upload a target list for the 2016 Performance Survey", minor = False)

Page [[User:Neil P. Quinn-WMF/01-AEAR]] saved
